In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

In [2]:
# Estabilish connection and load data into memory

spark = (SparkSession.builder
        .appName("Map")
        .getOrCreate())

# Get data
data = spark.sparkContext.textFile("/opt/homebrew/Cellar/apache-spark/3.3.1/README.md")

23/01/06 22:37:19 WARN Utils: Your hostname, Deboras-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.104 instead (on interface en0)
23/01/06 22:37:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/06 22:37:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Transformation

### map()

In [3]:
# Use map()
#For every line in the doc, create a tuple containing its content and lenght (content, lenght)
mapFile = data.map(lambda line : (line, len(line)))

# Print tuples
# foreach() is a action that starts the execution
mapFile.foreach(print)

('', 0)
('```python', 9)
('>>> spark.range(1000 * 1000 * 1000).count()', 43)
('```', 3)
('', 0)('# Apache Spark', 14)

('', 0)('## Example Programs', 19)
('', 0)
('Spark also comes with several sample programs in the `examples` directory.', 74)
('To run one of them, use `./bin/run-example <class> [params]`. For example:', 74)
('', 0)
('```bash', 7)
('./bin/run-example SparkPi', 25)
('```', 3)
('', 0)
('will run the Pi example locally.', 32)
('', 0)
('You can set the MASTER environment variable when running examples to submit', 75)

('examples to a cluster. This can be a mesos:// or spark:// URL,', 62)
('"yarn" to run on YARN, and "local" to run', 41)
('locally with one thread, or "local[N]" to run locally with N threads. You', 73)
('can also use an abbreviated class name if the class is in the `examples`', 72)
('Spark is a unified analytics engine for large-scale data processing. It provides', 80)('package. For instance:', 22)

('', 0)('high-level APIs in Scala, Java, Python, and R, an

### flatMap()

In [4]:
# Use flatMap()
# Split line into words
flatFile = data.flatMap(lambda line : line.split())

# Print words
flatFile.foreach(print)

#
Apache
Spark
Spark
is
a
unified
analytics
engine
for
large-scale
data
processing.
It
provides
high-level
APIs
in
Scala,
Java,
Python,
and
R,
and
an
optimized
engine
that
supports
general
computation
graphs
for
data
analysis.
It
also
supports
a
rich
set
of
higher-level
tools
including
Spark
SQL
for
SQL
and
DataFrames,
pandas
API
on
Spark
for
pandas
workloads,
MLlib
for
machine
learning,
GraphX
for
graph
processing,
and
Structured
Streaming
for
stream
processing.
<https://spark.apache.org/>
[![GitHub
Action
Build](https://github.com/apache/spark/actions/workflows/build_and_test.yml/badge.svg?branch=master&event=push)](https://github.com/apache/spark/actions/workflows/build_and_test.yml?query=branch%3Amaster+event%3Apush)
[![AppVeyor
Build](https://img.shields.io/appveyor/ci/ApacheSoftwareFoundation/spark/master.svg?style=plastic&logo=appveyor)](https://ci.appveyor.com/project/ApacheSoftwareFoundation/spark)
[![PySpark
Coverage](https://codecov.io/gh/apache/spark/branch/master/graph/bad

### filter(func)

In [5]:
# Use filter()
# Only words starts with "a"
filterFile = flatFile.filter(lambda word : word.startswith("a"))

# Print words
filterFile.foreach(print)

also
a
a
and
also
an
abbreviated
are
a
also
a
and
against
at
and
a
and
an
a
analytics
and
and
an
analysis.
also
a
and
and
a
and
a
available
at
an
also


### reduceByKey(func)

In [6]:
list = ["um", "um", "dois", "dois", "três"]

rdd = spark.sparkContext.parallelize(list) #transform list in RDD

# map list into tuple and use reduceByKey to count frequency of every word
rdd2 = rdd.map(lambda x: (x, 1)).reduceByKey(lambda a,b : a+b)

rdd2.foreach(print)

('três', 1)
('um', 2)
('dois', 2)


### sortByKey(func)

In [7]:
# map list into tuple, use reduceByKey to count frequency of every word and sort by key
rdd2 = rdd.map(lambda x: (x, 1)).reduceByKey(lambda a,b : a+b).sortByKey("asc")

rdd2.foreach(print)

('dois', 2)
('três', 1)
('um', 2)


### union(rdd)

In [8]:
list2 = ["um", "quatro", "cinco"]

rdd2 = spark.sparkContext.parallelize(list2)

rddUnion = rdd.union(rdd2)

rddUnion.foreach(print)

dois
um
três
um
dois
quatro
um
cinco


### intersection(rdd)

In [9]:
rddIntersection = rdd.intersection(rdd2)
rddIntersection.foreach(print)

um


### distinct(rdd)

In [10]:
rddDistinct = rdd.distinct()
rddDistinct.foreach(print)

um
dois
três


### join(rdd)

In [11]:
list = [("Pedro", 38), ("Maria", 42), ("João", 12)]
list2 = [("Pedro", "BH"), ("Maria", "DF")]

rdd = spark.sparkContext.parallelize(list)
rdd2 = spark.sparkContext.parallelize(list2)

rddJoin = rdd.join(rdd2)

rddJoin.foreach(print)

('Pedro', (38, 'BH'))
('Maria', (42, 'DF'))


## Action

### foreach(func)

In [12]:
list = [("Pedro", 38), ("Maria", 42), ("João", 12)]
list2 = [("Pedro", "BH"), ("Maria", "DF")]

rdd = spark.sparkContext.parallelize(list)
rdd2 = spark.sparkContext.parallelize(list2)

rddJoin = rdd.join(rdd2)
#rddJoin.foreach(print)

### collect()

In [13]:
print(rddJoin.collect())

[('Maria', (42, 'DF')), ('Pedro', (38, 'BH'))]


### count()

In [14]:
print(rddJoin.count())

2


### take(n)

In [15]:
# rddUnion.foreach(print)
rddUnion.take(4)

['um', 'um', 'dois', 'dois']

### top(k)

In [16]:
# rddUnion.foreach(print)
rddUnion.top(4)

['um', 'um', 'um', 'três']

### countByValue()

In [17]:
rddUnion.countByValue()

defaultdict(int, {'um': 3, 'dois': 2, 'três': 1, 'quatro': 1, 'cinco': 1})

### reduce(func)

In [18]:
rddUnion.reduce(lambda a,b : a + ' ' + b)

'um um dois dois três um quatro cinco'

### saveAsTextFile(path)

In [20]:
rddUnion.saveAsTextFile('/Users/deborabastos/Documents/5.Bootcamp DS/Módulo2/bootcamp_code/')

Py4JJavaError: An error occurred while calling o414.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/Users/deborabastos/Documents/5.Bootcamp DS/Módulo2/bootcamp_code already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:299)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:406)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1089)
	at org.